<a href="https://colab.research.google.com/github/KaustavRaj/Text-Summarization/blob/master/Evaluation_Metric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluation metric
---

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install contractions
!pip install rouge

In [4]:
import re
import os
import pickle
import logging
import numpy as np
import pandas as pd
import contractions
from nltk.corpus import stopwords
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)


class Summarizer:

  def __init__(self):
    self.dir_path = '/content/gdrive/My Drive/Colab Notebooks/Summarization/summarization v2'
    self._MAX_TEXT_LEN    =  60
    self._MAX_SUMMARY_LEN =  10
    self._TEXT_PADDING    =  'post'
    self.encoder_model = load_model(self.dir_path + '/models/encoder_model.h5')
    self.decoder_model = load_model(self.dir_path + '/models/decoder_model.h5')
    with open(self.dir_path + '/data/word_indices_mapping.pickle', 'rb') as f:
      self.index_to_word_text, self.index_to_word_summary, self.word_to_index_summary = pickle.load(f)
    with open(self.dir_path + '/data/tok_x.pickle', 'rb') as f:
      self.tok_x = pickle.load(f)


  def summarize(self, sent):
    """wrapper function to test the model"""

    sent = self.cleaner(sent, remove_stopwords=True)
    if len(sent.split()) > self._MAX_TEXT_LEN:
      return "make your sentence length less than {} words".format(self._MAX_TEXT_LEN)

    seq = self.tok_x.texts_to_sequences(sent.split())
    seq = [[item for sublist in seq for item in sublist]]
    seq = pad_sequences(seq, maxlen=self._MAX_TEXT_LEN, padding=self._TEXT_PADDING)
    return self.seq2seq_model(seq.reshape(1, self._MAX_TEXT_LEN))


  def cleaner(self, text, remove_stopwords=True):
    """removes url's, nltk's stopwords and anything which is not an alphabet"""

    stop_words = set(stopwords.words('english'))
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text.lower(), flags=re.MULTILINE)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = contractions.fix(text, slang=False)
    if remove_stopwords:
      text = ' '.join([word for word in text.split() if word not in stop_words]).strip()
      
    return text
  

  def seq2seq_model(self, input_seq):
    """summarizes the input text given and returns the summarized string"""

    encoder_out, encoder_h, encoder_c = self.encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = self.word_to_index_summary['stok']
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
      output_tokens, h, c = self.decoder_model.predict([target_seq] + [encoder_out, encoder_h, encoder_c])
      sampled_token_index = np.argmax(output_tokens[0, -1, :])
      sampled_token = self.index_to_word_summary[sampled_token_index]
      
      if sampled_token != 'etok':
        decoded_sentence += sampled_token + ' '
        
      if sampled_token == 'etok' or len(decoded_sentence.split()) >= (self._MAX_SUMMARY_LEN-1):
        stop_condition = True

      target_seq = np.zeros((1,1))
      target_seq[0, 0] = sampled_token_index
      encoder_h, encoder_c = h, c

    return decoded_sentence

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
dir_path = '/content/gdrive/My Drive/Colab Notebooks/Summarization/summarization v2'
df = pd.read_csv(dir_path + '/data/prepared_amazon_reviews_dataset.csv')[['text', 'summary']]

In [6]:
df.head()

,text,summary
0,bought several vitality canned dog food produc...,stok good quality dog food etok
1,product arrived labeled jumbo salted peanuts p...,stok not as advertised etok
2,confection around centuries light pillowy citr...,stok delight says it all etok
3,looking secret ingredient robitussin believe f...,stok cough medicine etok
4,great taffy great price wide assortment yummy ...,stok great taffy etok


In [0]:
df.dropna(inplace=True)

In [8]:
df.shape

(70707, 2)

In [0]:
# loading our model
model = Summarizer()

In [0]:
from rouge import Rouge

scorer = Rouge()

In [0]:
# storing all the rouge scores
rouge_1 = {'f':[], 'p':[], 'r':[]}
rouge_2 = {'f':[], 'p':[], 'r':[]}
rouge_l = {'f':[], 'p':[], 'r':[]}

In [0]:
for i in range(df.shape[0]):
  text = df['text'][i]
  reference = ' '.join(df['summary'][i].split()[1:-1])
  if len(reference) == 0:
    continue
  hypothesis = model.summarize(text)
  scores = scorer.get_scores(hypothesis, reference)[0]
  one, two, l = scores['rouge-1'], scores['rouge-2'], scores['rouge-l']
  for p, q in one.items():
    rouge_1[p].append(q)
  for p, q in two.items():
    rouge_2[p].append(q)
  for p, q in l.items():
    rouge_l[p].append(q)

Our average rouge scores are:

In [18]:
np.mean(rouge_1['f']), np.mean(rouge_1['p']), np.mean(rouge_1['r'])

(0.2784578212176004, 0.3258323772425466, 0.2658607938551489)

In [19]:
np.mean(rouge_2['f']), np.mean(rouge_2['p']), np.mean(rouge_2['r'])

(0.12287177605435542, 0.14134767031609272, 0.11870969176261821)

In [20]:
np.mean(rouge_l['f']), np.mean(rouge_l['p']), np.mean(rouge_l['r'])

(0.25834125883560194, 0.3227466068601844, 0.26382645947286665)